In [ ]:
###############################
#Started on Thu January 23 2020
#@author: Jonathan Frame
###############################
# This program is designed to generate a synthetic dataset of atmospheric forcing conditions that 
# match the NLDAS data from 1980 - 2014.
# The synthetic record can be extended out as far as the user likes,
# and the user can add nonstationary conditions by adjusting the Parameters controlling the distributions.

In [3]:
%%bash
cat test.conf
echo ""

# This is an example of an input file for MetSim
[MetSim]
#out_vars = ['temp', 'prec', 'shortwave', 'longwave', 'vapor_pressure', 'rel_humid', 'air_pressure', 'wind']

# Time step in minutes
time_step = 180

# Forcings begin here (year/month/day:hour) (hour optional)
start = 1950/1/1

# Forcings end at this date (year/month/day)
stop = 1950/1/31

# Input and output directories
forcing = ./input/cli_test.nc
domain  = ./input/cli_domain.nc
state =   ./input/cli_state.nc

forcing_fmt = netcdf

out_dir = ./output
output_prefix = test

prec_type = triangle

[chunks]
lat = 3
lon = 1

[forcing_vars]
prec  = Prec
t_max = Tmax
t_min = Tmin
wind  = wind

[state_vars]
prec  = prec
t_max = t_max
t_min = t_min

[domain_vars]
lat  = lat
lon  = lon
mask = mask
elev = elev
t_pk = t_pk
dur  = dur



In [5]:
%%bash
source activate metsim_whw
which ms

/home/jmframe/programs/anaconda3/envs/metsim_whw/bin/ms


In [11]:
%%bash
source activate metsim_whw
ms MetSim-tutorial-master/test.conf -v -n 2

/home/jmframe/programs/anaconda3/envs/metsim_whw/lib/python3.6/site-packages/metsim/metsim.py:205: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime instead.
  self.params[p] = pd.datetime(year, month, day)
Traceback (most recent call last):
  File "/home/jmframe/programs/anaconda3/envs/metsim_whw/bin/ms", line 10, in <module>
    sys.exit(main())
  File "/home/jmframe/programs/anaconda3/envs/metsim_whw/lib/python3.6/site-packages/metsim/cli/ms.py", line 99, in main
    ms.run()
  File "/home/jmframe/programs/anaconda3/envs/metsim_whw/lib/python3.6/site-packages/metsim/metsim.py", line 273, in run
    self.setup_netcdf_output(filename, times)
  File "/home/jmframe/programs/anaconda3/envs/metsim_whw/lib/python3.6/site-packages/metsim/metsim.py", line 304, in setup_netcdf_output
    with Dataset(filename, mode="w") as ncout:
  File "netCDF4/_netCDF4.pyx", line 2321, in netCDF4._netCDF4.Dataset.__init__
  File 

CalledProcessError: Command 'b'source activate metsim_whw\nms MetSim-tutorial-master/test.conf -v -n 2\n'' returned non-zero exit status 1.

In [ ]:
import numpy as np
import pandas as pd
import time
import datetime as dt
import sys 
import os
import matplotlib.pyplot as plt
import scipy.stats as ss
from metsim import MetSim

In [ ]:
# PROGRAM INITIALIZATION OPTIONS

ymi = 'yearmonth' # Year month index
#  Synthetic data generating parameters
OF = 2 # A multiplier (integer) for continuing out the synthetic forcing data record past the original NLDAS.
PRECIP_PARAMETER1 = 0.75 #  Synthetic data generating parameters
PRECIP_PARAMETER2 = 0.25
TEMP_PARM1 = 0.25
TEMP_PARM2 = 5.8
TEMP_PARM3 = 0.09
TEMP_PARM4 = 5.2
TEMP_PARM5 = 0.13
nGauges = 673 # Setting nGauges to be known number of gauges in CAMELS
count_basins = 0 # just a value to keep track of loops
max_basins = 1 # Stopping the loop after this many basins, for testing the script.
np.set_printoptions(threshold=sys.maxsize)# print options for generating the results files.

In [ ]:
#Store all attributes. Really just need basin area, but the rest might come in handy.
att_path = "/home/NearingLab/data/camels_attributes_v2.0/camels_all.txt"
attributes = pd.read_csv(att_path, sep=";")
attributes.set_index('gauge_id', inplace=True)
#data directory path
df_path = "/home/NearingLab/projects/jmframe/CAMELS_synthData/nldas_forcing/"

In [ ]:
dataSample = pd.read_csv(df_path+'13240000_lump_nldas_forcing_leap.txt',sep='\t| ',header=3, engine='python')
dataSample.head()

In [ ]:
count_basins = 0 # just a value to keep track of loops
# Loop through all the files in the forcing data directory.
for filename in os.listdir(df_path):
    print(filename)
    
    nforcing = './input/green_river_forcing.nc'
    ndomain = './input/green_river_domain.nc'
    nstate = './input/green_river_state.nc'
    
    dates = pd.date_range('1980/01/01', '1980/12/31')
    params = {
        'time_step'    : "1440",       
        'start'        : dates[0],
        'stop'         : dates[-1],
        'forcing'      : nforcing,     
        'domain'       : ndomain,
        'state'        : nstate,
        'forcing_fmt'  : 'netcdf',
        'out_dir'      : './output',
        'output_prefix': filename,
        'scheduler'    : 'threading',
        'chunks'       : {'hru': 3},
        'forcing_vars' : {'prcp' : 'prec', 'tmax': 't_max', 'tmin': 't_min'},
        'state_vars'   : {'prcp' : 'prec', 'tmax': 't_max', 'tmin': 't_min'},
        'domain_vars'  : {'elevation': 'elev', 'latitude': 'lat', 'longitude': 'lon', 'mask': 'mask'}
        }
    

    count_basins+=1
    if count_basins >= max_basins:
        break

In [ ]:
# This is an example of an input file for MetSim
[MetSim]

# Time step in minutes
time_step = 60

# Forcings begin here (year/month/day:hour)
start = 1950/1/1

# Forcings end at this date (year/month/day)
stop = 1950/1/31

# Input specification
forcing = ./tests/data/test.nc
domain  = ./tests/data/domain.nc
state = ./tests/data/state_nc.nc
in_fmt = netcdf

# Output specification
out_dir = ./results
out_prefix = forcing
out_precision = f8

[chunks]
lat = 3
lon = 3

[forcing_vars]
prec = Prec
t_max = Tmax
t_min = Tmin

[state_vars]
prec = prec
t_max = t_max
t_min = t_min

[domain_vars]
lat = lat
lon = lon
mask = mask
elev = elev